In [16]:
import os
import pandas as pd
from sklearn.model_selection import TimeSeriesSplit
# os.chdir('..')
print(os.getcwd())

c:\Users\User\Desktop\DS\Projects\oil-optimization


In [12]:
df = pd.read_csv('data/processed/features.csv', parse_dates=['period']).drop('year_month',axis=1)
df['cpi_energy'] = df['cpi_energy'].interpolate()
df['oil_production'] = df['oil_production'].interpolate()

,period,gasoline_price,Exports,Imports,East Coast (PADD 1),Midwest (PADD 2),Gulf Coast (PADD 3),Rocky Mountain (PADD 4),West Coast (PADD 5),us_dollar_index,volatility_index,cpi_energy,oil_production
0,2015-01-02,2.385552,388,6856,1158,3600,8643,579,2439,102.45182,17.293,199.926,581987.0
1,2015-01-09,2.300862,388,7492,1118,3517,8272,599,2388,103.36484,18.982,200.545,571924.4
2,2015-01-16,2.221552,388,7218,996,3179,7844,577,2312,103.17986,20.996,201.164,561861.8
3,2015-01-23,2.181103,502,7422,1104,3348,7857,581,2366,103.79498,18.444,201.783,551799.2
4,2015-01-30,2.194793,502,7387,1129,3618,7845,591,2361,104.79696,18.582,202.402,541736.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
560,2025-09-26,3.429483,3751,5833,744,4029,8616,638,2140,120.54606,16.190,281.120,855462.0
561,2025-10-03,3.421241,3590,6403,824,3982,8823,619,2048,120.60824,16.394,281.120,855462.0
562,2025-10-10,3.363000,4466,5525,828,3589,8271,575,1866,121.16382,17.600,281.120,855462.0
563,2025-10-17,3.338172,4203,5918,847,3635,8781,599,1870,121.32104,21.314,281.120,855462.0


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 565 entries, 0 to 564
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   period                   565 non-null    datetime64[ns]
 1   gasoline_price           565 non-null    float64       
 2   Exports                  565 non-null    int64         
 3   Imports                  565 non-null    int64         
 4   East Coast (PADD 1)      565 non-null    int64         
 5   Midwest (PADD 2)         565 non-null    int64         
 6   Gulf Coast (PADD 3)      565 non-null    int64         
 7   Rocky Mountain (PADD 4)  565 non-null    int64         
 8   West Coast (PADD 5)      565 non-null    int64         
 9   us_dollar_index          565 non-null    float64       
 10  volatility_index         565 non-null    float64       
 11  year_month               565 non-null    object        
 12  cpi_energy               129 non-nul

In [ ]:
df['period'] = pd.to_datetime(df['period'])

In [17]:
df

,period,gasoline_price,Exports,Imports,East Coast (PADD 1),Midwest (PADD 2),Gulf Coast (PADD 3),Rocky Mountain (PADD 4),West Coast (PADD 5),us_dollar_index,volatility_index,cpi_energy,oil_production
0,2015-01-02,2.385552,388,6856,1158,3600,8643,579,2439,102.45182,17.293,199.926,581987.0
1,2015-01-09,2.300862,388,7492,1118,3517,8272,599,2388,103.36484,18.982,200.545,571924.4
2,2015-01-16,2.221552,388,7218,996,3179,7844,577,2312,103.17986,20.996,201.164,561861.8
3,2015-01-23,2.181103,502,7422,1104,3348,7857,581,2366,103.79498,18.444,201.783,551799.2
4,2015-01-30,2.194793,502,7387,1129,3618,7845,591,2361,104.79696,18.582,202.402,541736.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
560,2025-09-26,3.429483,3751,5833,744,4029,8616,638,2140,120.54606,16.190,281.120,855462.0
561,2025-10-03,3.421241,3590,6403,824,3982,8823,619,2048,120.60824,16.394,281.120,855462.0
562,2025-10-10,3.363000,4466,5525,828,3589,8271,575,1866,121.16382,17.600,281.120,855462.0
563,2025-10-17,3.338172,4203,5918,847,3635,8781,599,1870,121.32104,21.314,281.120,855462.0


### Feature engineering
* Splitting
* Scaling
* Filling nulls